# Processing Non-Genetic Data

This code was run in the UK Biobank Research Analysis Platform to process operation and diagnosis codes for UKB, create overarching CVD event and date variables, denote an individual’s status at endline (dead, CVD, or censored), and add covariates such as the location of the assessment center, date of assessment, smoking, and educational attainment. We also created BMI as the exposure variable, which was based on the first measured BMI, the time that also marked an individual’s start date of follow up. 

In [ ]:
dx download "ALASTCOHORT.csv"


In [ ]:
# Installing relevant packages
install.packages('stringr')
library(stringr)

install.packages("lubridate")
library(lubridate)

In [ ]:
UKBDF <- read.csv("ALASTCOHORT.csv")
colnames(UKBDF)
ncol(UKBDF)

In [ ]:
# First subsetting to only surgeries to keep relevant ones and dates
UKBDFOPS <- UKBDF[ , c(2, 101:225)]


In [ ]:
# Using p41272 - Operative procedures with OPCS4 Codes to restrict to relevant surgeries
UKBDFOPS$RELEVANTOP <- grepl('K40|K41|K42|K43|K44|K45|K46', UKBDFOPS$p41272)
# Creates T/F variable for whether operation contained is in our criteria


In [ ]:
# Only extract string BEFORE operation - subsequent operations aren't relevant (first diagnosis/surgery)
UKBDFOPS$p41272 <- sub("K40.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K41.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K42.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K43.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K44.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K45.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K46.*", "", UKBDFOPS$p41272)


In [ ]:
# ONLY record as non-missing if individual has record of correct operation
UKBDFOPS$V3 <- ifelse(is.na(UKBDFOPS$RELEVANTOP) == FALSE, str_count(UKBDFOPS$p41272, ','), NA)
# This code works for selecting array because FIRST array is actually a0 NOT a1


In [ ]:
# Using number of commas found BEFORE operation to choose array
UKBDFOPS$V4 <- paste("p41282_a", UKBDFOPS$V3, sep = "")


In [ ]:
# Choosing column based on V4
UKBDFOPS$MATCH <- match(UKBDFOPS$V4, names(UKBDFOPS))


In [ ]:
# Only keeping where not missing
UKBDFOPS <- subset(UKBDFOPS, RELEVANTOP == TRUE)


In [ ]:
# Large for loop
OPDATE <- matrix(NA, nrow = nrow(UKBDFOPS), ncol = 1)
# Using this to create a variable that tracks the date of relevant operations

# Create as separate data frame and then join back after creating OPDATE var
for(i in 1:nrow(UKBDFOPS)){
  OPDATE[i, ] <- UKBDFOPS[i, UKBDFOPS$MATCH[i]]
  
}

UKBDFOPS$OperationDate <- OPDATE

In [ ]:
# Creating region variables
UKBDFMerge$REGION <- ifelse(UKBDFMerge$p54_i0 == "Cardiff" | UKBDFMerge$p54_i0 == "Swansea" | UKBDFMerge$p54_i0 == "Wrexham", "Wales",
                            ifelse(UKBDFMerge$p54_i0 == "Edinburgh" | UKBDFMerge$p54_i0 == "Glasgow", "Scotland", "England"))

summary(as.factor(UKBDFMerge$REGION))



In [ ]:
# ADDING EID BACK IN AND SEE HOW THIS WORKS WITH EVERYTHING....
varnames <- c("eid","REGION","p22001", "p31", "p191", "p21022", "p52", "p34", "p738_i0", "p738_i1", "p738_i2", "p738_i3",
              "p6138_i0", "p6138_i1", "p6138_i2", "p6138_i3", "p20160_i0", "p20160_i1", "p20160_i2",
              "p20160_i3", "p20116_i0", "p20116_i1", "p20116_i2", "p20116_i3", "p3894_i0", "p3894_i1",
              "p3894_i2", "p3894_i3", "p3627_i0", "p3627_i1", "p3627_i2", "p3627_i3", "p4056_i0", "p4056_i1",
              "p4056_i2", "p4056_i3", "p21001_i0", "p21001_i1", "p21001_i2", "p21001_i3",
              "p23104_i0","p23104_i1", "p23104_i2", "p23104_i3", "p48_i0", "p48_i1", "p48_i2", "p48_i3",
              "p49_i0", "p49_i1", "p49_i2", "p49_i3", "p40000_i0", "p40000_i1", "p40007_i0", "p40007_i1",
              "p40001_i0", "p40001_i1", "p42000", "p42006", "p131296", "p131297", "p131298", "p131299",
              "p131300", "p131301", "p131302", "p131303", "p131304", "p131305", "p131306", "p131307",
              "p131342", "p131343", "p131354", "p131355", "p131360", "p131361", "p131362", "p131363",
              "p131364", "p131365", "p131366", "p131367", "p131368", "p131369", "p131378", "p131379",
              "p53_i0", "p53_i1", "p53_i2", "p53_i3", "p21003_i0", "p21003_i1", "p21003_i2", "p21003_i3","p9999990","p999999")



# Getting ordered alpha numerically
# Remove p to sort numerically then add back in
varnames <- gsub("p", "", varnames)
varnames <- varnames[order(as.numeric(varnames))]
varnames <- paste("p", varnames, sep = "")

In [ ]:
# Remove extraneous region variabless
UKBDFNoEID <- UKBDFMerge[ , -c(2, 97:225)]


In [ ]:
# Renaming the last two
colnames(UKBDFNoEID)[97] <- "p999999" #OperationDate
colnames(UKBDFNoEID)[98] <- "p9999990" # REGION

# Renaming EID AS WELL
colnames(UKBDFNoEID)[1] <- "p9999999999" # EID


In [ ]:
# First remove p
colnames(UKBDFNoEID) <- gsub("p", "", colnames(UKBDFNoEID))

# Now order numerically
UKBDFNoEID <- UKBDFNoEID[ , order(as.numeric(colnames(UKBDFNoEID)))]

# Adding p back in
colnames(UKBDFNoEID) <- paste("p", colnames(UKBDFNoEID), sep = "")

# Renaming fields to be interpretable - after organizing numerically
colnames(UKBDFNoEID) <- c("Biological Sex", "Year of Birth", "Month of Birth", "Date Lost to Follow Up",
                          "Age at Recruitment", "Genetic Sex", "Date of MI", "Date of Stroke", "Date of I20",
                          "Source of I20", "Date of I21", "Source of I21", "Date of I22", "Source of I22", "Date of I23", "Source of I23",
                          "Date of I24", "Source of I24", "Date of I25", "Source of I25", "Date of I44", "Source of I44",
                          "Date of I50", "Source of I50", "Date of I60", "Source of I60", "Date of I61", "Source of I61",
                          "Date of I62", "Source of I62", "Date of I63", "Source of I63", "Date of I64", "Source of I64",
                          "Date of I69", "Source of I69", "OperationDate", "REGION", "ID", "HH Income Inst 0", "HH Income Inst 1", "HH Income Inst 2", "HH Income Inst 3",
                          "EA Inst 0", "EA Inst 1", "EA Inst 2", "EA Inst 3", "EverSmoke Inst 0", "EverSmoke Inst 1", "EverSmoke Inst 2",
                          "EverSmoke Inst 3", "SmokStat Inst 0", "SmokStat Inst 1", "SmokStat Inst 2", "SmokStat Inst 3", "Age Heart Attack Inst 0",
                          "Age Heart Attack Inst 1", "Age Heart Attack Inst 2", "Age Heart Attack Inst 3", "Age Angina Inst 0", "Age Angina Inst 1",
                          "Age Angina Inst 2", "Age Angina Inst 3", "Age Stroke Inst 0", "Age Stroke Inst 1", "Age Stroke Inst 2", "Age Stroke Inst 3",
                          "Measured BMI Inst 0", "Measured BMI Inst 1", "Measured BMI Inst 2", "Measured BMI Inst 3", "Imped BMI Inst 0", "Imped BMI Inst 1",
                          "Imped BMI Inst 2", "Imped BMI Inst 3", "WC Inst 0", "WC Inst 1", "WC Inst 2", "WC Inst 3", "HC Inst 0", "HC Inst 1", "HC Inst 2", "HC Inst 3",
                          "Date of Death Inst 0", "Date of Death Inst 1", "Age at Death Inst 0", "Age at Death Inst 1", "Primary Cause of Death Inst 0", "Primary Cause of Death Inst 1",
                          "Date Attending Assess Center Inst 0", "Date Attending Assess Center Inst 1", "Date Attending Assess Center Inst 2", "Date Attending Assess Center Inst 3",
                          "Age Attending Assess Center Inst 0", "Age Attending Assess Center Inst 1", "Age Attending Assess Center Inst 2", "Age Attending Assess Center Inst 3")



In [ ]:

# Making date of birth variable
# ONLY take last two variables of Year
UKBDFNoEID$Year <- sprintf('%02d', UKBDFNoEID$`Year of Birth` %% 100)
UKBDFNoEID$Month <- match(UKBDFNoEID$`Month of Birth`, month.name)

# Convert into DOB - arbitrarily chose first of the month
UKBDFNoEID$DOB <- as.Date(paste(UKBDFNoEID$`Year of Birth`, UKBDFNoEID$Month, 01, sep = "-"))



UKBDFNoEID$DOB

In [ ]:
# NOW processing ICD-10 codes (derived from hospital, death, self-report, etc.)
# CVD Variables here include:
# 7, 8, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 106, 105, 104
UKBDFNoEID$CVDDate <- apply(UKBDFNoEID[ , c(7, 8, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 106, 105, 104)], 1, FUN = min, na.rm = TRUE)
# Excluded my 3 user created ones for now... Not much info and simpler...
# 47,238 PREVALENT CVD
# 229,003 CONTROLS
colnames(UKBDFNoEID)
ncol(UKBDFNoEID)

# Create Status variable denoting whether someone did or did not have a CVD event
UKBDFNoEID$Status <- ifelse(is.na(UKBDFNoEID$CVDDate) == FALSE, 1, 0)

summary(as.factor(UKBDFNoEID$Status))


In [ ]:

# Make sure these dates attending assessment center are coded as dates
UKBDFNoEID$`Date Attending Assess Center Inst 0` <- as.Date(UKBDFNoEID$`Date Attending Assess Center Inst 0`)
UKBDFNoEID$`Date Attending Assess Center Inst 1` <- as.Date(UKBDFNoEID$`Date Attending Assess Center Inst 1`)
UKBDFNoEID$`Date Attending Assess Center Inst 2` <- as.Date(UKBDFNoEID$`Date Attending Assess Center Inst 2`)
UKBDFNoEID$`Date Attending Assess Center Inst 3` <- as.Date(UKBDFNoEID$`Date Attending Assess Center Inst 3`)

# First record of measured BMI comes from first appearance at assessment center
UKBDFNoEID$BMIDate <- apply(UKBDFNoEID[ , c(90:93)], 1, FUN = min, na.rm = TRUE)

# Converting to BMI date
UKBDFNoEID$BMIDate <- as.Date(UKBDFNoEID$BMIDate)
summary(UKBDFNoEID$BMIDate)

# Time at risk starts after BMI is measured for the first time
UKBDFNoEID$Time <- UKBDFNoEID$CVDDate - UKBDFNoEID$BMIDate


# Drop people with NO BMIDate (No BMI to measure...)
UKBDFNoEID <- subset(UKBDFNoEID, is.na(BMIDate) == FALSE)

nrow(UKBDFNoEID)
# No one is missing BMI measurement!!!


# Create time to event variable
UKBDFNoEID$Time <- UKBDFNoEID$CVDDate - UKBDFNoEID$BMIDate


# Creating Age at Baseline (first measured BMI/WHRadjBMI)
UKBDFNoEID$AgeBaseline <- UKBDFNoEID$BMIDate - UKBDFNoEID$DOB

# Convert this from days to years
UKBDFNoEID$AgeBaseline <- UKBDFNoEID$AgeBaseline/365.24


summary(as.numeric(UKBDFNoEID$AgeBaseline))


# Keep if time is positive OR MISSING CVDDate (these would be control folks so far)
UKBDFNoEID <- subset(UKBDFNoEID, Time > 0 | is.na(CVDDate) == TRUE)


nrow(UKBDFNoEID)
# 256,982 individuals now
# So 19,259 inds excluded due to prevalent CVD

summary(as.factor(UKBDFNoEID$Status))
# Suggests INCIDENT CVD is STILL 27,979
# Controls of 229,003


# Choosing actual BMI measurement based on date attended assessment center
UKBDFNoEID$FirstBMI <- ifelse(UKBDFNoEID$BMIDate == UKBDFNoEID$`Date Attending Assess Center Inst 0`, UKBDFNoEID$`Measured BMI Inst 0`,
                              ifelse(UKBDFNoEID$BMIDate == UKBDFNoEID$`Date Attending Assess Center Inst 1`, UKBDFNoEID$`Measured BMI Inst 1`,
                                     ifelse(UKBDFNoEID$BMIDate == UKBDFNoEID$`Date Attending Assess Center Inst 2`, UKBDFNoEID$`Measured BMI Inst 2`, UKBDFNoEID$`Measured BMI Inst 3`)))
# TURNED THIS INTO AN IF ELSE STATEMENT IN NOTEBOOK

In [ ]:

# --------
# MAKING Death Variable with Timing
# MAKING lost to follow up variable (only a few...) - only around 1300 in UKB
# IF NEITHER DEATH NOR CVD, THEN CENSORED at 2022 OR AT LOSS TO FOLLOW UP
# --------

UKBDFNoEID$`Date of Death Inst 0` <- as.Date(UKBDFNoEID$`Date of Death Inst 0`)
UKBDFNoEID$`Date of Death Inst 1` <- as.Date(UKBDFNoEID$`Date of Death Inst 1`)

# Assuming CENSORED at 1/1/22.. Not quite right - will vary by locale apparently
UKBDFNoEID$CensorDate <- as.Date("2020-01-01")

# Finishing Time Variable based on WHEN things happened
# IF not dead, CVD, or CENSORED (ignore) then time out is 2022
# Differential follow up was added later
# o.w. time is CVD -> Dead -> Censored
UKBDFNoEID$Time <- ifelse(is.na(UKBDFNoEID$CVDDate) == FALSE, UKBDFNoEID$Time, 
                          ifelse(is.na(UKBDFNoEID$`Date of Death Inst 0`) == FALSE, UKBDFNoEID$`Date of Death Inst 0` - UKBDFNoEID$BMIDate, 
                                 ifelse(is.na(UKBDFNoEID$`Date of Death Inst 1`) == FALSE, UKBDFNoEID$`Date of Death Inst 1` - UKBDFNoEID$BMIDate, 
                                        UKBDFNoEID$CensorDate - UKBDFNoEID$BMIDate)))


# Update Status Variable accordingly
UKBDFNoEID$Status <- ifelse(is.na(UKBDFNoEID$CVDDate) == FALSE, UKBDFNoEID$Status, 
                            ifelse(is.na(UKBDFNoEID$`Date of Death Inst 0`) == FALSE | is.na(UKBDFNoEID$`Date of Death Inst 1`) == FALSE, 2, 0))

summary(as.factor(UKBDFNoEID$Status))
# 217,836 inds censored
# 27,979 CVD
# 11,167 Dead

# -------
# Format EVER SMOKED correctly
# -------

UKBDFNoEID$`EverSmoke Inst 0` == UKBDFNoEID$`EverSmoke Inst 1`
UKBDFNoEID$`EverSmoke Inst 1` == UKBDFNoEID$`EverSmoke Inst 2`
UKBDFNoEID$`EverSmoke Inst 2` == UKBDFNoEID$`EverSmoke Inst 3`


# Getting Relevant EA - not missing and equivalent ACROSS WAVES
UKBDFNoEID$EverSmoke <- ifelse(is.na(UKBDFNoEID$`EverSmoke Inst 0`) == FALSE, UKBDFNoEID$`EverSmoke Inst 0`,
                               ifelse(is.na(UKBDFNoEID$`EverSmoke Inst 1`) == FALSE, UKBDFNoEID$`EverSmoke Inst 1`,
                                      ifelse(is.na(UKBDFNoEID$`EverSmoke Inst 2`) == FALSE, UKBDFNoEID$`EverSmoke Inst 2`,
                                             ifelse(is.na(UKBDFNoEID$`EverSmoke Inst 3`) == FALSE, UKBDFNoEID$`EverSmoke Inst 3`, NA))))




# --------
# Get Educational Attainment right
# --------

# Getting Relevant EA - not missing and equivalent ACROSS WAVES
UKBDFNoEID$EA <- ifelse(is.na(UKBDFNoEID$`EA Inst 0`) == FALSE, UKBDFNoEID$`EA Inst 0`,
                        ifelse(is.na(UKBDFNoEID$`EA Inst 1`) == FALSE, UKBDFNoEID$`EA Inst 1`,
                               ifelse(is.na(UKBDFNoEID$`EA Inst 2`) == FALSE, UKBDFNoEID$`EA Inst 2`,
                                      ifelse(is.na(UKBDFNoEID$`EA Inst 3`) == FALSE, UKBDFNoEID$`EA Inst 3`, NA))))


UKBDFLowSES <- UKBDFNoEID[grep("None of the above", UKBDFNoEID$EA), ]
UKBDFHighSES <- UKBDFNoEID[-grep("None of the above", UKBDFNoEID$EA), ]


# -------
# Getting cases and controls count OVERALL then by HIGHER VS LOWER EA
# -------

summary(as.factor(UKBDFNoEID$Status))
# 217,836 censored, 27,979 CVD, 11,167 dead
http://localhost:8888/notebooks/Processing%20Non-Genetic%20Data.ipynb#

summary(as.factor(UKBDFLowSES$Status))
# 29,215 censored, 6,749 CVD, 2,703 dead

summary(as.factor(UKBDFHighSES$Status))
# 188,621 censored, 21,230 CVD, 8,464 dead

# Write these into CSVs
write.csv(UKBDFNoEID, "FULLCSV.csv")
write.csv(UKBDFLowSES, "LOWSESCSV.csv")
write.csv(UKBDFHighSES, "HIGHSESCSV.csv")

In [ ]:
dx upload FULLCSV.csv
dx upload HIGHSESCSV.csv
dx upload LOWSESCSV.csv